In [ ]:
from itertools import chain
from pathlib import Path

from IPython.display import Markdown, display
from tqdm.notebook import tqdm

import hashlib
from os import environ
import subprocess

from githelpers import open_pbcopy
import ghapicache
import release_notes

%load_ext autoreload
%autoreload 1
%aimport githelpers
%aimport ghapicache
%aimport release_notes

In [35]:
import githelpers
cached = ghapicache.GhApiCache(repo="g4vg")
githelpers.REPO = Path.home() / "Code/g4vg"
ghapi = cached.api

In [3]:
# Load teams
TEAMS = {}
for t in tqdm(cached.teams()):
    TEAMS[t['name']] = release_notes.get_team(cached, t)

team_members = frozenset(chain(*(t.members for t in TEAMS.values())))
org_members = frozenset(m['login'] for m in cached.org_members())
TEAMS['affiliates'] = release_notes.Team(description="Associated with Celeritas but not core members",
                                         members=(org_members - team_members))

# Initialize the user cache
local_repo = Path("/Users/seth/Code/celeritas-temp")
user_cache = release_notes.UserCache(cached, local_repo / "scripts/release/users.json")
for login in tqdm(org_members):
    user_cache[login]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

# Draft github release

In [33]:
release_md = release_notes.ReleaseMetadata(
    release='1.0.5',
    merge_bases=['v1.0.4'],
    target_branch='main'
)

In [ ]:
ghapi = cached.api

gh_release = release_notes.find_release(ghapi, release_md.release)

In [16]:
tarball = release_notes.get_or_upload_tarball(cached, gh_release)
print(f"Artifact URL: {tarball.url}")

No tarball found in release assets: reloading from GitHub
Loading https://github.com/celeritas-project/g4vg/releases/download/v1.0.5/g4vg-1.0.5.tar.gz from cached file data/ghapicache-downloads/93c98ff870315508e9521256a83f0c7f92b2cc7e.gz
Found tarball in release assets
Artifact URL: https://github.com/celeritas-project/g4vg/releases/download/v1.0.5/g4vg-1.0.5.tar.gz


In [23]:
# assert 0
sha256_hash = hashlib.sha256(tarball.content).hexdigest()
with open_pbcopy() as pb:
    pb.write(f'version("{release_md.release}", sha256="{sha256_hash}")\n')
print("Spack version copied to clipboard!")
spack = Path(environ["SPACK_ROOT"]) / "bin" / "spack"
package_dir = Path(
    subprocess.check_output([spack, "location", "-p", cached.repo]).decode().strip()
)
subprocess.check_call(["open", package_dir / "package.py"])

Spack version copied to clipboard!


0

# Push to zenodo

In [24]:
import zenodoapi
%aimport zenodoapi

In [25]:
# Load the Zenodo token
token_path = Path.home() / ".config/zenodo-token"
with open(token_path) as f:
    zenodo_token = f.read().strip()

zenodo = zenodoapi.Zenodo(zenodo_token)
zenodo.api_url = "https://zenodo.org/api/"

In [26]:
def load_contributions(release_md):
    prs = release_notes.PullRequestRange(release_md)
    count_contrib = release_notes.ContributionCounter(cached)
    for pr in tqdm(prs.pull_ids):
        count_contrib(pr)

    # Create author list
    return count_contrib.sorted()

def download_tarball(gh_release):
    asset = gh_release['assets'][0]
    name = asset['name']
    content = cached.download_file(asset['url'], ext=Path(name).suffix)
    assert content is not None
    return (content, name)

make_zenodo_md = release_notes.ZenodoMetadataBuilder(user_cache=user_cache, teams=TEAMS)

## Minor release

This assumes the major release has already been published upstream into Zenodo.

In [27]:
assert not release_md.is_major()
major, minor, patch = release_md.as_version()
release_md = release_notes.ReleaseMetadata.from_comprehensive_version(minor, patch)
major_dep = zenodo.find_deposition(f"G4VG {major}.{minor}")

In [29]:
zenodo.get_deposition(major_dep.get_latest_version().id)

ZenodoDeposition(id=15450422, title="G4VG 1.0")

In [ ]:
gh_release = release_notes.find_release(ghapi, release_md.release)

In [38]:
# Get the Zenodo metadata    
from requests import HTTPError


old_md = major_dep.data["metadata"]
new_md = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
# Only update editors, not team members
old_contrib = new_md["contributors"]
new_contrib = [u for u in old_contrib if u["type"] == "Editor"]
old_contrib = [u for u in old_contrib if u["type"] != "Editor"]
new_md["contributors"] = new_contrib + old_contrib
# Don't change the title
new_md["title"] = major_dep.data["metadata"]["title"]
# Add the body
new_md["description"] = "\n\n".join(
    [
        old_md["description"],
        f"<h1>Version {release_md.release}</h1>",
        new_md["description"],
    ]
)
try:
    new_vers = major_dep.create_new_version()
except HTTPError as e:
    # Maybe the version already exists
    if e.response.status_code == 400:
        print("Draft may already exist:", e)
        new_vers = major_dep.get_latest_draft()
    else:
        raise
try:
    new_vers.update(new_md)
except HTTPError as e:
    # Something else went wrong? Try to upload the tarball
        print("Draft may already exist:", e)

# Upload the release
(content, name) = download_tarball(gh_release)
new_vers.upload(content, name)
# The old tarball may still be there (this is buggy) so delete it
new_vers.refresh()
for file in new_vers.get_files():
    if file.filename != name:
        try:
            file.delete()
        except Exception as e:
            print(f"Failed to delete {file.filename}: {e}")
new_vers.refresh()

  0%|          | 0/4 [00:00<?, ?it/s]

Created new version 16949066: https://zenodo.org/deposit/16949066
Updated deposition at https://zenodo.org/deposit/16949066 : G4VG 1.0
Uploaded g4vg-1.0.5.tar.gz: version b6bfa1a8-a628-4ace-97db-a75f2120baa1
Failed to delete g4vg-1.0.4.tar.gz: Expecting value: line 1 column 1 (char 0)


ZenodoDeposition(id=16949066, title="G4VG 1.0")

## Major release

In [57]:
assert release_md.is_major()
major, minor, patch = release_md.as_version()
release_md = release_notes.ReleaseMetadata.from_comprehensive_version(minor)
title = f"Celeritas {major}.{minor}"

In [58]:
zmd = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
zmd["title"] = title

# Create the Zenodo deposition
deposition = zenodo.create_deposition(zmd)

  0%|          | 0/234 [00:00<?, ?it/s]

Created deposition 15281110 at https://zenodo.org/deposit/15281110 : Celeritas 0.6


In [60]:
# Upload the release tarball
deposition.upload(artifact_tgz, f"celeritas-{release_md.release}.tar.gz")

print(deposition.html)
print("""
IMPORTANT Checklist:
- Add community (CANNOT be done later)
- Update contributors based on release date
- Update licenses
""")

Uploaded celeritas-0.6.0.tar.gz: version bbd63016-30a1-4be9-9a74-329ca975bed9
https://zenodo.org/deposit/15281110

IMPORTANT Checklist:
- Add community (CANNOT be done later)
- Update contributors based on release date
- Update licenses

